In [90]:
# Demonstration of online linear regression.

# Python 3.8.10
import numpy as np # Test in version 1.21.1.
import random
from sklearn import datasets, linear_model # sklearn v. 0.24.2
from sklearn.metrics import mean_squared_error, r2_score
from scipy import linalg # Scipy 1.7.0
import sympy # 1.8

In [74]:
# Online linear regression with a single feature is demonstrated here:
# https://stackoverflow.com/questions/52070293/efficient-online-linear-regression-algorithm-in-python
# Following is a direct lifting of the code in the first answer
# Note: if all values of new_x in the first array are the same, the result is a divide-by-zero error.

def lr(x_avg,y_avg,Sxy,Sx,n,new_x,new_y):
    """
    x_avg: average of previous x, if no previous sample, set to 0
    y_avg: average of previous y, if no previous sample, set to 0
    Sxy: covariance of previous x and y, if no previous sample, set to 0
    Sx: variance of previous x, if no previous sample, set to 0
    n: number of previous samples
    new_x: new incoming 1-D numpy array x
    new_y: new incoming 1-D numpy array x
    """
    new_n = n + len(new_x)

    new_x_avg = (x_avg*n + np.sum(new_x))/new_n
    new_y_avg = (y_avg*n + np.sum(new_y))/new_n

    if n > 0:
        x_star = (x_avg*np.sqrt(n) + new_x_avg*np.sqrt(new_n))/(np.sqrt(n)+np.sqrt(new_n))
        y_star = (y_avg*np.sqrt(n) + new_y_avg*np.sqrt(new_n))/(np.sqrt(n)+np.sqrt(new_n))
    elif n == 0:
        x_star = new_x_avg
        y_star = new_y_avg
    else:
        raise ValueError

    new_Sx = Sx + np.sum((new_x-x_star)**2)
    new_Sxy = Sxy + np.sum((new_x-x_star).reshape(-1) * (new_y-y_star).reshape(-1))

    beta = new_Sxy/new_Sx
    alpha = new_y_avg - beta * new_x_avg
    return new_Sxy, new_Sx, new_n, alpha, beta, new_x_avg, new_y_avg

# Example of online linear regression applied to 101 batches of random data.
x_avg, y_avg, Sxy, Sx, n = 0,0,0,0,0
random.seed(1234)
X = np.array([random.random() for i in range(10)])
y = np.array([random.random() + 5*X[i] for i in range(10)])

X_total = X
y_total = y

Sxy, Sx, n, alpha, beta, x_avg, y_avg = lr(x_avg,y_avg,Sxy,Sx,n, X,y)

for i in range(100):
    X = np.array([random.random() for i in range(10)])
    X_total = np.append(X_total, X)
    y = np.array([random.random() + 5*X[i] for i in range(10)])
    y_total = np.append(y_total, y)
    Sxy, Sx, n, alpha, beta, x_avg, y_avg = lr(x_avg,y_avg,Sxy,Sx,n, X,y)
    
# Results. alpha and beta are, respectively, the intercept and coefficient of the regression.
Sxy, Sx, n, alpha, beta, x_avg, y_avg

(416.15301008042337,
 82.8002593290743,
 1010,
 0.49901136647134914,
 5.025986795844447,
 0.506835402394537,
 3.046359406572799)

In [3]:
# Use scikit learn's linear model to validate the above algorithm
# The intercept and coefficient should match the alpha and beta values, respectively, found above.
regr = linear_model.LinearRegression()
regr.fit(X_total.reshape(-1,1), y_total)
[regr.intercept_,regr.coef_]

[0.4990113664713478, array([5.0259868])]

In [83]:
# Online multidimensional regression:
# The following is based on a formula for regression coefficient, given for example in the following:
# https://stattrek.com/multiple-regression/regression-coefficients.aspx

In [75]:
# The core of this notebook: implementation of online regression.
# There are n training examples with p features each.
# If results are calculated once and n>p, then total time is O(np^2 + p^3).
# If results are calculated once for every training example, then time is O(np^3).
# For complexity purposes, the sizes of data batches is irrelevant.

def lr_multi(XX,Xy,X,y, calc_results=False):
    # Time complexity analysis is noted. XX is a (p+1)X(p+1) matrix and Xy is a vector of p+1 numbers.
    # Overall complexity is O(np^2).
    XX = np.add(XX, np.matmul(X.transpose(),X))
    Xy = np.add(Xy, np.matmul(X.transpose(),y))
    if (calc_results):
        # We are assuming O(p^3) complexity for row reduced echelon form, as floating point operations are employed.
        # See also this discussion:
        # https://cstheory.stackexchange.com/questions/3921/what-is-the-actual-time-complexity-of-gaussian-elimination
        lin_ind_cols = sympy.Matrix(XX).T.rref()[1]
        # O(p^2) space and time to reduce XX and Xy to linearly independent features
        XX_reduced = [
                [XX[i][j] for j in range(len(XX[0])) if j in lin_ind_cols]
            for i in range(len(XX)) if i in lin_ind_cols]
        Xy_reduced = [[Xy[i][0]] for i in range(len(XX)) if i in lin_ind_cols]
        # XX_reduced is of size O(p).
        # Matrix inversion takes O(p^3) time under Gauss-Jordan.
        # See https://en.wikipedia.org/wiki/Computational_complexity_of_mathematical_operations
        # Faster algorithms may be possible.
        # Matrix multiplication is O(p^2) under naive approaches.
        result = np.matmul( np.linalg.inv(XX_reduced), Xy_reduced )
        # Rebuild the full results from the above results with only linear independent features.
        # Should be O(p) time to do the following.
        full_result = np.zeros((len(XX)))
        result_pointer = 0
        for i in range(len(XX)):
            if i in lin_ind_cols:
                full_result[i] = result[result_pointer]
                result_pointer += 1
        return XX, Xy, {"intercept":full_result[0], "coefficients":full_result[1:]}
    else:
        return XX, Xy, None

In [80]:
# Example for testing
def example():
    y_base = np.array([10,11,12,7,7])
    X_base = np.array([[1,2],[0,1],[3,5],[2,1],[3,3]])
    y = y_base.reshape(1,len(y_base)).transpose()
    X = np.concatenate(([[1]]*len(X_base),X_base), axis=1)
    XX = np.zeros( ( len(X[0]) , len(X[0]) ) )
    Xy = np.zeros( ( len(X[0]) , 1 ) )

    # Split into 2
    X1, X2, y1, y2 = X[:3], X[3:], y[:3], y[3:]
    XX, Xy, _ = lr_multi(XX,Xy,X1,y1)
    XX, Xy, results = lr_multi(XX,Xy,X2,y2, True)
    print(results)

    # Validate with scikit learn. Should match the intercept and coefficients found above.
    regr = linear_model.LinearRegression()
    regr.fit(X_base, y_base)
    print({"intercept":regr.intercept_, "coefficients":regr.coef_})
example()

{'intercept': 9.04545454545456, 'coefficients': array([-2.27272727,  1.85227273])}
{'intercept': 9.045454545454543, 'coefficients': array([-2.27272727,  1.85227273])}


In [81]:
# Another example. The matrix XX in the algorithm is singular.

def singular_example():
    y_base = np.array([10,11,12,7,7])
    # Note that the second column is twice the first, making it redundant for regression purposes.
    X_base = np.array([[1,2],[2,4],[1,2],[3,6],[1,2]]) 
    y = y_base.reshape(1,len(y_base)).transpose()
    X = np.concatenate(([[1]]*len(X_base),X_base), axis=1)
    XX = np.zeros( ( len(X[0]) , len(X[0]) ) )
    Xy = np.zeros( ( len(X[0]) , 1 ) )

    X1,X2,y1,y2 = X[:2],X[2:],y[:2],y[2:]

    XX, Xy, _ = lr_multi(XX,Xy,X1,y1)
    XX, Xy, results = lr_multi(XX,Xy,X2,y2,True)
    print(results)
    # These results are not the same as the results given by scikit-learn's regression.
    # In the event of the X^TX matrix being singular (linear dependency among features), there is not an unambiguous
    # regression that minimizes error.
    regr = linear_model.LinearRegression()
    regr.fit(X_base, y_base)
    print({"intercept":regr.intercept_, "coefficients":regr.coef_})
singular_example()

{'intercept': 11.0, 'coefficients': array([-1.,  0.])}
{'intercept': 11.0, 'coefficients': array([-0.2, -0.4])}


In [67]:
def large_example():
    # A larger example
    y_base = np.random.rand(1000)
    X_base = np.random.rand(1000,10)
    
    y = y_base.reshape(1,len(y_base)).transpose()
    X = np.concatenate(([[1]]*len(X_base),X_base), axis=1)
    XX = np.zeros( ( len(X[0]) , len(X[0]) ) )
    Xy = np.zeros( ( len(X[0]) , 1 ) )
    
    breakpoints = [0,300,500,800,1000]
    for i in range(len(breakpoints)-2):
        XX, Xy, _ = lr_multi(XX,Xy,X[breakpoints[i]:breakpoints[i+1]], y[breakpoints[i]:breakpoints[i+1]])
    _,_,results = lr_multi(XX,Xy,X[breakpoints[-2]:], y[breakpoints[-2]:],True)
    print(results)
    
    # Validate with scikit-learn's linear regression.
    regr = linear_model.LinearRegression()
    regr.fit(X_base, y_base)
    print({"intercept":regr.intercept_, "coefficients":regr.coef_})
results = large_example()


{'intercept': 0.42971638609834667, 'coefficients': array([-0.00111586,  0.02239917,  0.02781732, -0.01231627,  0.07486401,
       -0.01070935,  0.0487162 , -0.01034687,  0.01303653,  0.00970912])}
{'intercept': 0.4297163860983432, 'coefficients': array([-0.00111586,  0.02239917,  0.02781732, -0.01231627,  0.07486401,
       -0.01070935,  0.0487162 , -0.01034687,  0.01303653,  0.00970912])}


In [77]:
# Example of a situation where all inputs are identical.
# In that case, it is impossible to learn any coefficient weights, so the result is merely an intercept
# that is the average of all responses.
def degenerate_example():
    y_base = np.array([1,2,3,4,5])
    X_base = np.array([[1,1],[1,1],[1,1],[1,1],[1,1]])
    
    y = y_base.reshape(1,len(y_base)).transpose()
    X = np.concatenate(([[1]]*len(X_base),X_base), axis=1)
    XX = np.zeros( ( len(X[0]) , len(X[0]) ) )
    Xy = np.zeros( ( len(X[0]) , 1 ) )
    _,_,results = lr_multi(XX,Xy,X,y,True)
    print(results)
    
    # Validate with scikit-learn's linear regression.
    regr = linear_model.LinearRegression()
    regr.fit(X_base, y_base)
    print({"intercept":regr.intercept_, "coefficients":regr.coef_})
    
degenerate_example()

{'intercept': 3.0, 'coefficients': array([0., 0.])}
{'intercept': 3.0, 'coefficients': array([0., 0.])}
